In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from joblib import dump

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import ModelCheckpoint
from tensorflow import set_random_seed
set_random_seed(12)

Using TensorFlow backend.
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaco

# Preprocessing

In [2]:
data = pd.read_csv('./datasets/fundamental_data.csv')

In [3]:
y_data = data['growth']
data.drop(columns='growth')

X_train, X_test, Y_train, Y_test = train_test_split(data, y_data, test_size=0.2, random_state=12)

In [4]:
# Normalise data
normaliser = preprocessing.MinMaxScaler()
X_train = normaliser.fit_transform(X_train)
X_test = normaliser.transform(X_test)

# Y normaliser
y_normaliser = preprocessing.MinMaxScaler()
Y_train = y_normaliser.fit_transform(Y_train.to_numpy().reshape(-1, 1))
Y_test = y_normaliser.transform(Y_test.to_numpy().reshape(-1, 1))

In [5]:
# Save scalers for future use
dump(normaliser, './normalisers/fundamental_x_normaliser.joblib')
dump(y_normaliser, './normalisers/fundamental_y_normaliser.joblib')

['./normalisers/fundamental_y_normaliser.joblib']

# Model selection

##### Model constructor

In [6]:
def build_dense(x, y, batch_size=512, epochs=24, **params):
    
    # List of parameters
    if 'density' not in params: params['density'] = x.shape[1]
    if 'activation' not in params: params['activation'] = 'relu'
    if 'optimizer' not in params: params['optimizer'] = 'adam'
    if 'shuffle' not in params: params['shuffle'] = False
    
    # Model definition
    model = Sequential()
    
    model.add(Dense(params['density'], activation=params['activation']))
    
    density = params['density']//2
    while density >= 12:
        if 'dropout' in params:
            model.add(Dropout(params['dropout']))
        model.add(Dense(density, activation=params['activation']))
        density //= 2
            
    model.add(Dense(1, activation='linear'))
    
    model.compile(loss='mse', optimizer=params['optimizer'])
    
    if 'callbacks' in params:
        callback = model.fit(x=x, y=y, validation_split=0.1, batch_size=batch_size, epochs=epochs, shuffle=params['shuffle'],
                            callbacks=params['callbacks'])
    else:
        callback = model.fit(x=x, y=y, validation_split=0.1, batch_size=batch_size, epochs=epochs, shuffle=params['shuffle'])
    
    return [model, callback, params]

##### Evolutive algorith to search for the most optimal model

In [7]:
def set_genes(x, y, population_size, population=[]):
    if population_size-len(population) < 0:
        print('population size must be greater or equal than actual population')
        return
    
    if len(population) > 0:
        if len(population) < 3:
            print('population should be empty or 3 at least')
            return
        
        population[2] = breed_genes(x, y, population[0][2].copy())
        population[1] = breed_genes(x, y, combine=(population[0][2], population[1][2]))
        population = population[:3]
    
    for _ in range(population_size-len(population)):
        subject = breed_genes(x, y)
        population.append(subject)
        
    return population

def breed_genes(x, y, genes={}, combine=None):
    genes['x'] = x
    genes['y'] = y
    
    if type(combine) is list or type(combine) is tuple:
        genes['density'] = combine[np.random.randint(0,2)]['density']
        genes['activation'] = combine[np.random.randint(0,2)]['activation']
        genes['optimizer'] = combine[np.random.randint(0,2)]['optimizer']
        genes['shuffle'] = combine[np.random.randint(0,2)]['shuffle']
        
        genes['dropout'] = combine[np.random.randint(0,2)].get('dropout')
        if genes['dropout'] is None: del genes['dropout']
            
    else:
        if np.random.randint(0,3) == 1:
            genes['density'] = int((np.random.randint(x.shape[1]//2,x.shape[1]*2.66)//2)*2)

        if np.random.randint(0,3) == 1:
            activation = [
                'relu',
                'sigmoid',
                'softplus',
                'softsign',
                'tanh',
                'selu',
                'elu',
                'exponential'
            ]
            genes['activation'] = activation[np.random.randint(0,8)]

        if np.random.randint(0,3) == 1:
            optimizer = [
                'sgd',
                'rmsprop',
                'adam',
                'adadelta',
                'adagrad',
                'adamax',
                'nadam'
            ]
            genes['optimizer'] = optimizer[np.random.randint(0,7)]

        if np.random.randint(0,3) == 1:
            genes['shuffle'] = True

        if np.random.randint(0,3) == 1:
            genes['dropout'] = np.random.randint(1,4)/10
            
    new_model = build_dense(**genes)
        
    return new_model

In [8]:
def breed_population(x, y, generations, population_size, population=[]):
    
    if generations > 1 and population_size < 3:
        print('population size should be of a minimum of 3 for more than one generation')
        return
    
    for g in range(generations):
        print(f'\nGENERATION {g}\n')
        population = set_genes(x, y, population_size, population)
        population = sorted(population, key=lambda x: x[1].history['val_loss'][-1]+x[1].history['loss'][-1])
        
    return population

In [9]:
best_models = breed_population(X_train, Y_train, generations=20, population_size=10)


GENERATION 0

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 21471 samples, validate on 2386 samples
Epoch 1/24
21471/21471 [==============================] - 1s 36us/step - loss: 0.0107 - val_loss: 8.8871e-04
Epoch 2/24
21471/21471 [==============================] - 0s 5us/step - loss: 0.0023 - val_loss: 2.9519e-04
Epoch 3/24
21471/21471 [==============================] - 0s 6us/step - loss: 0.0012 - val_loss: 9.2847e-05
Epoch 4/24
21471/21471 [==============================] - 0s 6us/step - loss: 7.7633e-04 - val_loss: 8.4896e-05
Epoch 5/24
21471/21471 [==============================] - 0s 6us/step - loss: 5.4246e-04 - val_loss: 1.1669e-04
Epoch 6/24
21471/21471 [==============================] - 0s 6us/step - loss: 4.2114e-04 - val_loss: 5.5622e-05
Epoch 7/24
21471/21471 [==============================] - 0s 6us/step - loss: 3.2186e-04 - val_loss: 4.1438e-05
Epoch 8/24
21471/21471 [===================

21471/21471 [==============================] - 0s 5us/step - loss: 31719061455.7653 - val_loss: 5802.2791
Epoch 19/24
21471/21471 [==============================] - 0s 5us/step - loss: 2515862864.4995 - val_loss: 5802.2388
Epoch 20/24
21471/21471 [==============================] - 0s 5us/step - loss: 31616295895.5957 - val_loss: 5802.2309
Epoch 21/24
21471/21471 [==============================] - 0s 5us/step - loss: 41379940663.5961 - val_loss: 5802.2283
Epoch 22/24
21471/21471 [==============================] - 0s 5us/step - loss: 127730688280551.5938 - val_loss: 5802.2162
Epoch 23/24
21471/21471 [==============================] - 0s 5us/step - loss: 1857303876614.8938 - val_loss: 5802.2136
Epoch 24/24
21471/21471 [==============================] - 0s 5us/step - loss: 16562235844995282.0000 - val_loss: 5802.3340
Train on 21471 samples, validate on 2386 samples
Epoch 1/24
21471/21471 [==============================] - 0s 18us/step - loss: nan - val_loss: nan
Epoch 2/24
21471/21471 [===

21471/21471 [==============================] - 0s 6us/step - loss: 1.5670e-04 - val_loss: 3.2889e-05
Epoch 21/24
21471/21471 [==============================] - 0s 6us/step - loss: 1.5085e-04 - val_loss: 2.9829e-05
Epoch 22/24
21471/21471 [==============================] - 0s 6us/step - loss: 1.4298e-04 - val_loss: 3.1339e-05
Epoch 23/24
21471/21471 [==============================] - 0s 6us/step - loss: 1.3619e-04 - val_loss: 3.4512e-05
Epoch 24/24
21471/21471 [==============================] - 0s 6us/step - loss: 1.2376e-04 - val_loss: 3.1188e-05
Train on 21471 samples, validate on 2386 samples
Epoch 1/24
21471/21471 [==============================] - 0s 20us/step - loss: 0.1611 - val_loss: 0.0034
Epoch 2/24
21471/21471 [==============================] - 0s 6us/step - loss: 0.0233 - val_loss: 2.3594e-04
Epoch 3/24
21471/21471 [==============================] - 0s 6us/step - loss: 0.0130 - val_loss: 1.6796e-04
Epoch 4/24
21471/21471 [==============================] - 0s 6us/step - loss:

21471/21471 [==============================] - 0s 6us/step - loss: 0.0015 - val_loss: 5.9170e-05
Epoch 22/24
21471/21471 [==============================] - 0s 6us/step - loss: 0.0014 - val_loss: 5.6442e-05
Epoch 23/24
21471/21471 [==============================] - 0s 6us/step - loss: 0.0012 - val_loss: 4.8366e-05
Epoch 24/24
21471/21471 [==============================] - 0s 6us/step - loss: 0.0011 - val_loss: 5.8181e-05
Train on 21471 samples, validate on 2386 samples
Epoch 1/24
21471/21471 [==============================] - 1s 26us/step - loss: 0.0420 - val_loss: 4.8011e-04
Epoch 2/24
21471/21471 [==============================] - 0s 7us/step - loss: 0.0134 - val_loss: 1.4486e-04
Epoch 3/24
21471/21471 [==============================] - 0s 7us/step - loss: 0.0096 - val_loss: 2.8343e-04
Epoch 4/24
21471/21471 [==============================] - 0s 7us/step - loss: 0.0069 - val_loss: 2.3938e-04
Epoch 5/24
21471/21471 [==============================] - 0s 7us/step - loss: 0.0052 - val_los

21471/21471 [==============================] - 0s 6us/step - loss: 3.1351e-04 - val_loss: 4.8472e-05
Epoch 22/24
21471/21471 [==============================] - 0s 6us/step - loss: 2.7968e-04 - val_loss: 4.2204e-05
Epoch 23/24
21471/21471 [==============================] - 0s 6us/step - loss: 2.6678e-04 - val_loss: 4.5531e-05
Epoch 24/24
21471/21471 [==============================] - 0s 6us/step - loss: 2.6125e-04 - val_loss: 4.7397e-05
Train on 21471 samples, validate on 2386 samples
Epoch 1/24
21471/21471 [==============================] - 1s 29us/step - loss: 0.0900 - val_loss: 0.0060
Epoch 2/24
21471/21471 [==============================] - 0s 7us/step - loss: 0.0160 - val_loss: 0.0012
Epoch 3/24
21471/21471 [==============================] - 0s 7us/step - loss: 0.0088 - val_loss: 7.5131e-04
Epoch 4/24
21471/21471 [==============================] - 0s 7us/step - loss: 0.0059 - val_loss: 4.7162e-04
Epoch 5/24
21471/21471 [==============================] - 0s 7us/step - loss: 0.0043 -

...

21471/21471 [==============================] - 7s 314us/step - loss: 0.0149 - val_loss: 8.6595e-05
Epoch 2/24
21471/21471 [==============================] - 0s 18us/step - loss: 0.0037 - val_loss: 1.7390e-04
Epoch 3/24
21471/21471 [==============================] - 0s 18us/step - loss: 0.0021 - val_loss: 1.3610e-04
Epoch 4/24
21471/21471 [==============================] - 0s 18us/step - loss: 0.0013 - val_loss: 5.2684e-05
Epoch 5/24
21471/21471 [==============================] - 0s 18us/step - loss: 9.4021e-04 - val_loss: 4.5090e-05
Epoch 6/24
21471/21471 [==============================] - 0s 18us/step - loss: 6.9732e-04 - val_loss: 3.1924e-05
Epoch 7/24
21471/21471 [==============================] - 0s 18us/step - loss: 5.5608e-04 - val_loss: 4.0853e-05
Epoch 8/24
21471/21471 [==============================] - 0s 18us/step - loss: 4.5101e-04 - val_loss: 3.5556e-05
Epoch 9/24
21471/21471 [==============================] - 0s 18us/step - loss: 3.8727e-04 - val_loss: 3.1877e-05
Epoch 10/

21471/21471 [==============================] - 7s 348us/step - loss: nan - val_loss: nan
Epoch 2/24
21471/21471 [==============================] - 0s 21us/step - loss: nan - val_loss: nan
Epoch 3/24
21471/21471 [==============================] - 0s 21us/step - loss: nan - val_loss: nan
Epoch 4/24
21471/21471 [==============================] - 0s 22us/step - loss: nan - val_loss: nan
Epoch 5/24
21471/21471 [==============================] - 0s 21us/step - loss: nan - val_loss: nan
Epoch 6/24
21471/21471 [==============================] - 0s 21us/step - loss: nan - val_loss: nan
Epoch 7/24
21471/21471 [==============================] - 0s 21us/step - loss: nan - val_loss: nan
Epoch 8/24
21471/21471 [==============================] - 0s 22us/step - loss: nan - val_loss: nan
Epoch 9/24
21471/21471 [==============================] - 0s 22us/step - loss: nan - val_loss: nan
Epoch 10/24
21471/21471 [==============================] - 0s 23us/step - loss: nan - val_loss: nan
Epoch 11/24
21471/2

Epoch 9/24
21471/21471 [==============================] - 0s 21us/step - loss: 4.8075e-04 - val_loss: 5.1397e-05
Epoch 10/24
21471/21471 [==============================] - 0s 21us/step - loss: 4.1921e-04 - val_loss: 4.7056e-05
Epoch 11/24
21471/21471 [==============================] - 0s 20us/step - loss: 3.6986e-04 - val_loss: 3.8159e-05
Epoch 12/24
21471/21471 [==============================] - ETA: 0s - loss: 3.3669e-0 - 0s 20us/step - loss: 3.3340e-04 - val_loss: 3.7095e-05
Epoch 13/24
21471/21471 [==============================] - 0s 20us/step - loss: 3.0529e-04 - val_loss: 3.1505e-05
Epoch 14/24
21471/21471 [==============================] - 0s 20us/step - loss: 2.7844e-04 - val_loss: 3.9520e-05
Epoch 15/24
21471/21471 [==============================] - 0s 21us/step - loss: 2.6078e-04 - val_loss: 3.1184e-05
Epoch 16/24
21471/21471 [==============================] - 0s 20us/step - loss: 2.4425e-04 - val_loss: 3.1286e-05
Epoch 17/24
21471/21471 [==============================] - 0s

Epoch 9/24
21471/21471 [==============================] - 0s 19us/step - loss: 0.0010 - val_loss: 9.0804e-05
Epoch 10/24
21471/21471 [==============================] - 0s 19us/step - loss: 8.8266e-04 - val_loss: 7.6343e-05
Epoch 11/24
21471/21471 [==============================] - 0s 20us/step - loss: 7.8371e-04 - val_loss: 7.4253e-05
Epoch 12/24
21471/21471 [==============================] - 0s 19us/step - loss: 7.0408e-04 - val_loss: 5.4207e-05
Epoch 13/24
21471/21471 [==============================] - 0s 19us/step - loss: 6.1824e-04 - val_loss: 4.8643e-05
Epoch 14/24
21471/21471 [==============================] - 0s 19us/step - loss: 5.7375e-04 - val_loss: 5.2206e-05
Epoch 15/24
21471/21471 [==============================] - 0s 19us/step - loss: 5.1888e-04 - val_loss: 4.8722e-05
Epoch 16/24
21471/21471 [==============================] - 0s 19us/step - loss: 4.7108e-04 - val_loss: 4.5571e-05
Epoch 17/24
21471/21471 [==============================] - 0s 19us/step - loss: 4.2957e-04 - 

Epoch 9/24
21471/21471 [==============================] - 0s 19us/step - loss: 6.0789e-04 - val_loss: 9.9793e-05
Epoch 10/24
21471/21471 [==============================] - 0s 20us/step - loss: 5.4911e-04 - val_loss: 8.6194e-05
Epoch 11/24
21471/21471 [==============================] - 0s 19us/step - loss: 4.7701e-04 - val_loss: 8.2780e-05
Epoch 12/24
21471/21471 [==============================] - 0s 19us/step - loss: 4.3564e-04 - val_loss: 9.7710e-05
Epoch 13/24
21471/21471 [==============================] - 0s 19us/step - loss: 3.9361e-04 - val_loss: 7.4224e-05
Epoch 14/24
21471/21471 [==============================] - 0s 20us/step - loss: 3.5943e-04 - val_loss: 8.0051e-05
Epoch 15/24
21471/21471 [==============================] - 0s 19us/step - loss: 3.2913e-04 - val_loss: 8.1034e-05
Epoch 16/24
21471/21471 [==============================] - 0s 19us/step - loss: 3.0819e-04 - val_loss: 8.2839e-05
Epoch 17/24
21471/21471 [==============================] - 0s 20us/step - loss: 2.8356e-0

In [10]:
best_models[0][1].history['val_loss']

[0.00014578905102220337,
 5.324250623964949e-05,
 4.012273837487687e-05,
 3.502018207944752e-05,
 3.330115027713889e-05,
 3.511103870187986e-05,
 3.3601509853392936e-05,
 2.7667172159514137e-05,
 2.8690636981069248e-05,
 2.5271549512266405e-05,
 4.0283382234985905e-05,
 2.5433455736536672e-05,
 2.1910266929349953e-05,
 2.060753085101556e-05,
 2.013804340335699e-05,
 1.9526027911167032e-05,
 1.901653664551944e-05,
 1.8019791273405066e-05,
 1.7642038334318516e-05,
 1.6626616602857138e-05,
 1.8475294844359375e-05,
 1.5493819311466513e-05,
 1.4336143881158039e-05,
 1.4400636469889324e-05]

In [11]:
print('optimal parameters')
print('_______________________')
for key,val in best_models[0][2].items():
    if key != 'x' and key != 'y': print(f'{key}: {val}')

optimal parameters
_______________________
density: 72
activation: softsign
shuffle: True
dropout: 0.2
optimizer: adam


In [12]:
best_models[0][0].summary()

Model: "sequential_75"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_355 (Dense)            (None, 72)                7272      
_________________________________________________________________
dropout_207 (Dropout)        (None, 72)                0         
_________________________________________________________________
dense_356 (Dense)            (None, 36)                2628      
_________________________________________________________________
dropout_208 (Dropout)        (None, 36)                0         
_________________________________________________________________
dense_357 (Dense)            (None, 18)                666       
_________________________________________________________________
dense_358 (Dense)            (None, 1)                 19        
Total params: 10,585
Trainable params: 10,585
Non-trainable params: 0
_________________________________________________

In [13]:
checkpoint = ModelCheckpoint(
    filepath='./checkpoints/fundamental.{epoch}-{val_loss}.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

In [14]:
params = best_models[0][2]
params['x'] = X_train
params['y'] = Y_train
params['epochs'] = 2000
params['callbacks'] = [checkpoint]

In [15]:
final_model = build_dense(**params)

Train on 21471 samples, validate on 2386 samples
Epoch 1/2000
21471/21471 [==============================] - 7s 349us/step - loss: 0.0201 - val_loss: 1.9579e-04
Epoch 2/2000
21471/21471 [==============================] - 0s 20us/step - loss: 0.0043 - val_loss: 2.0349e-04
Epoch 3/2000
21471/21471 [==============================] - 0s 20us/step - loss: 0.0024 - val_loss: 1.2635e-04
Epoch 4/2000
21471/21471 [==============================] - 0s 20us/step - loss: 0.0015 - val_loss: 1.0517e-04
Epoch 5/2000
21471/21471 [==============================] - 0s 20us/step - loss: 0.0011 - val_loss: 6.7393e-05
Epoch 6/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.5102e-04 - val_loss: 4.7601e-05
Epoch 7/2000
21471/21471 [==============================] - 0s 20us/step - loss: 6.7104e-04 - val_loss: 4.0687e-05
Epoch 8/2000
21471/21471 [==============================] - 0s 21us/step - loss: 5.3943e-04 - val_loss: 3.9438e-05
Epoch 9/2000
21471/21471 [========================

21471/21471 [==============================] - 0s 20us/step - loss: 1.9430e-05 - val_loss: 5.9507e-06
Epoch 72/2000
21471/21471 [==============================] - 0s 21us/step - loss: 2.2248e-05 - val_loss: 6.6931e-06
Epoch 73/2000
21471/21471 [==============================] - 0s 20us/step - loss: 3.6722e-05 - val_loss: 5.9565e-06
Epoch 74/2000
21471/21471 [==============================] - 0s 20us/step - loss: 2.0112e-05 - val_loss: 5.9132e-06
Epoch 75/2000
21471/21471 [==============================] - 0s 21us/step - loss: 1.8780e-05 - val_loss: 5.8688e-06
Epoch 76/2000
21471/21471 [==============================] - 0s 20us/step - loss: 2.1776e-05 - val_loss: 5.8686e-06
Epoch 77/2000
21471/21471 [==============================] - 0s 21us/step - loss: 1.4328e-05 - val_loss: 5.9164e-06
Epoch 78/2000
21471/21471 [==============================] - 0s 22us/step - loss: 2.5380e-05 - val_loss: 6.0477e-06
Epoch 79/2000
21471/21471 [==============================] - 0s 22us/step - loss: 3.40

21471/21471 [==============================] - 0s 20us/step - loss: 1.0393e-05 - val_loss: 5.9448e-06
Epoch 142/2000
21471/21471 [==============================] - ETA: 0s - loss: 2.0592e-0 - 0s 20us/step - loss: 1.9147e-05 - val_loss: 6.3311e-06
Epoch 143/2000
21471/21471 [==============================] - 0s 20us/step - loss: 1.4681e-05 - val_loss: 5.0482e-06
Epoch 144/2000
21471/21471 [==============================] - 0s 20us/step - loss: 1.1403e-05 - val_loss: 4.6080e-06
Epoch 145/2000
21471/21471 [==============================] - 0s 20us/step - loss: 1.2006e-05 - val_loss: 6.3057e-06
Epoch 146/2000
21471/21471 [==============================] - 0s 20us/step - loss: 1.9227e-05 - val_loss: 5.6504e-06
Epoch 147/2000
21471/21471 [==============================] - 0s 21us/step - loss: 1.8090e-05 - val_loss: 4.4126e-06
Epoch 148/2000
21471/21471 [==============================] - 0s 21us/step - loss: 1.3190e-05 - val_loss: 7.6917e-06
Epoch 149/2000
21471/21471 [=======================

Epoch 211/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.5568e-05 - val_loss: 2.5310e-05
Epoch 212/2000
21471/21471 [==============================] - 0s 20us/step - loss: 7.8061e-05 - val_loss: 2.3819e-05
Epoch 213/2000
21471/21471 [==============================] - 0s 20us/step - loss: 7.5537e-05 - val_loss: 2.2236e-05
Epoch 214/2000
21471/21471 [==============================] - 0s 20us/step - loss: 8.8072e-05 - val_loss: 2.5118e-05
Epoch 215/2000
21471/21471 [==============================] - 0s 20us/step - loss: 6.9488e-05 - val_loss: 2.0613e-05
Epoch 216/2000
21471/21471 [==============================] - 0s 20us/step - loss: 6.3723e-05 - val_loss: 1.7277e-05
Epoch 217/2000
21471/21471 [==============================] - 0s 20us/step - loss: 6.0129e-05 - val_loss: 1.6297e-05
Epoch 218/2000
21471/21471 [==============================] - 0s 21us/step - loss: 5.9820e-05 - val_loss: 1.4884e-05
Epoch 219/2000
21471/21471 [==============================] - 0s

21471/21471 [==============================] - 0s 20us/step - loss: 6.9750e-06 - val_loss: 1.9152e-06
Epoch 282/2000
21471/21471 [==============================] - 0s 20us/step - loss: 6.5520e-06 - val_loss: 2.3086e-06
Epoch 283/2000
21471/21471 [==============================] - 0s 20us/step - loss: 1.8952e-05 - val_loss: 3.5061e-06
Epoch 284/2000
21471/21471 [==============================] - 0s 21us/step - loss: 1.2000e-05 - val_loss: 3.0891e-06
Epoch 285/2000
21471/21471 [==============================] - 0s 20us/step - loss: 1.2001e-05 - val_loss: 2.1717e-06
Epoch 286/2000
21471/21471 [==============================] - 0s 20us/step - loss: 9.2318e-06 - val_loss: 1.7234e-06
Epoch 287/2000
21471/21471 [==============================] - 0s 21us/step - loss: 1.1076e-05 - val_loss: 2.9199e-06
Epoch 288/2000
21471/21471 [==============================] - 0s 21us/step - loss: 1.2224e-05 - val_loss: 1.4060e-06
Epoch 289/2000
21471/21471 [==============================] - 0s 20us/step - lo

...

21471/21471 [==============================] - 0s 20us/step - loss: 2.5029e-06 - val_loss: 3.7590e-07
Epoch 1672/2000
21471/21471 [==============================] - 0s 20us/step - loss: 2.4020e-06 - val_loss: 5.2177e-07
Epoch 1673/2000
21471/21471 [==============================] - 0s 21us/step - loss: 1.9412e-06 - val_loss: 5.0313e-07
Epoch 1674/2000
21471/21471 [==============================] - 0s 20us/step - loss: 2.1439e-06 - val_loss: 4.3068e-07
Epoch 1675/2000
21471/21471 [==============================] - 0s 20us/step - loss: 2.4198e-06 - val_loss: 6.2511e-07
Epoch 1676/2000
21471/21471 [==============================] - 0s 20us/step - loss: 2.4974e-06 - val_loss: 3.6616e-07
Epoch 1677/2000
21471/21471 [==============================] - 0s 20us/step - loss: 2.9631e-06 - val_loss: 2.9404e-07
Epoch 1678/2000
21471/21471 [==============================] - 0s 21us/step - loss: 1.2703e-06 - val_loss: 3.1418e-07
Epoch 1679/2000
21471/21471 [==============================] - 0s 20us/s

21471/21471 [==============================] - 0s 20us/step - loss: 1.6286e-06 - val_loss: 3.7041e-07
Epoch 1741/2000
21471/21471 [==============================] - 0s 20us/step - loss: 3.3131e-06 - val_loss: 1.1522e-06
Epoch 1742/2000
21471/21471 [==============================] - 0s 20us/step - loss: 2.2589e-06 - val_loss: 8.2258e-07
Epoch 1743/2000
21471/21471 [==============================] - 0s 20us/step - loss: 4.7513e-06 - val_loss: 1.0745e-06
Epoch 1744/2000
21471/21471 [==============================] - 0s 21us/step - loss: 2.0904e-06 - val_loss: 6.9891e-07
Epoch 1745/2000
21471/21471 [==============================] - 0s 20us/step - loss: 1.3209e-06 - val_loss: 6.7272e-07
Epoch 1746/2000
21471/21471 [==============================] - 0s 21us/step - loss: 2.6512e-06 - val_loss: 9.5363e-07
Epoch 1747/2000
21471/21471 [==============================] - 0s 20us/step - loss: 2.5103e-06 - val_loss: 8.7718e-07
Epoch 1748/2000
21471/21471 [==============================] - 0s 20us/s

21471/21471 [==============================] - 0s 20us/step - loss: 2.7866e-06 - val_loss: 1.5410e-06
Epoch 1810/2000
21471/21471 [==============================] - 0s 20us/step - loss: 1.9066e-06 - val_loss: 1.9070e-06
Epoch 1811/2000
21471/21471 [==============================] - 0s 20us/step - loss: 2.5838e-06 - val_loss: 1.2821e-06
Epoch 1812/2000
21471/21471 [==============================] - 0s 20us/step - loss: 1.8799e-06 - val_loss: 1.0614e-06
Epoch 1813/2000
21471/21471 [==============================] - 0s 20us/step - loss: 2.2606e-06 - val_loss: 7.6761e-07
Epoch 1814/2000
21471/21471 [==============================] - 0s 20us/step - loss: 2.3414e-06 - val_loss: 7.4908e-07
Epoch 1815/2000
21471/21471 [==============================] - 0s 20us/step - loss: 4.5938e-06 - val_loss: 8.3949e-07
Epoch 1816/2000
21471/21471 [==============================] - 0s 20us/step - loss: 5.9122e-06 - val_loss: 1.0705e-06
Epoch 1817/2000
21471/21471 [==============================] - 0s 20us/s

21471/21471 [==============================] - 0s 21us/step - loss: 2.5779e-06 - val_loss: 2.9626e-06
Epoch 1879/2000
21471/21471 [==============================] - 0s 23us/step - loss: 2.1060e-06 - val_loss: 1.0126e-06
Epoch 1880/2000
21471/21471 [==============================] - 0s 20us/step - loss: 2.7709e-06 - val_loss: 7.2380e-06
Epoch 1881/2000
21471/21471 [==============================] - 0s 20us/step - loss: 4.0493e-06 - val_loss: 6.7272e-07
Epoch 1882/2000
21471/21471 [==============================] - 0s 20us/step - loss: 2.6775e-06 - val_loss: 7.6476e-07
Epoch 1883/2000
21471/21471 [==============================] - 0s 20us/step - loss: 1.7002e-06 - val_loss: 1.2766e-06
Epoch 1884/2000
21471/21471 [==============================] - 0s 20us/step - loss: 2.6973e-06 - val_loss: 8.2619e-07
Epoch 1885/2000
21471/21471 [==============================] - 0s 20us/step - loss: 3.3133e-06 - val_loss: 5.2129e-07
Epoch 1886/2000
21471/21471 [==============================] - 0s 20us/s

21471/21471 [==============================] - 0s 21us/step - loss: 4.7760e-06 - val_loss: 7.2600e-07
Epoch 1948/2000
21471/21471 [==============================] - 0s 21us/step - loss: 1.7407e-06 - val_loss: 5.6233e-07
Epoch 1949/2000
21471/21471 [==============================] - 0s 20us/step - loss: 1.7802e-06 - val_loss: 4.8895e-07
Epoch 1950/2000
21471/21471 [==============================] - 0s 20us/step - loss: 2.3003e-06 - val_loss: 4.6170e-07
Epoch 1951/2000
21471/21471 [==============================] - 0s 22us/step - loss: 2.3881e-06 - val_loss: 5.7069e-07
Epoch 1952/2000
21471/21471 [==============================] - 0s 22us/step - loss: 1.8703e-06 - val_loss: 5.4494e-07
Epoch 1953/2000
21471/21471 [==============================] - 0s 20us/step - loss: 3.6732e-06 - val_loss: 4.0385e-06
Epoch 1954/2000
21471/21471 [==============================] - 0s 20us/step - loss: 2.9235e-06 - val_loss: 1.0585e-06
Epoch 1955/2000
21471/21471 [==============================] - 0s 21us/s

In [16]:
final_model[2]

{'density': 72,
 'activation': 'softsign',
 'shuffle': True,
 'dropout': 0.2,
 'optimizer': 'adam',
 'callbacks': [<keras.callbacks.callbacks.ModelCheckpoint at 0x203e5d342c8>]}

In [17]:
final_model[0].summary()

Model: "sequential_182"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_832 (Dense)            (None, 72)                7272      
_________________________________________________________________
dropout_470 (Dropout)        (None, 72)                0         
_________________________________________________________________
dense_833 (Dense)            (None, 36)                2628      
_________________________________________________________________
dropout_471 (Dropout)        (None, 36)                0         
_________________________________________________________________
dense_834 (Dense)            (None, 18)                666       
_________________________________________________________________
dense_835 (Dense)            (None, 1)                 19        
Total params: 10,585
Trainable params: 10,585
Non-trainable params: 0
________________________________________________

In [18]:
final_model[0].load_weights('./checkpoints/fundamental.(best).hdf5')
y_predicted = final_model[0].predict(X_test)

In [19]:
predicted_y_test = y_normaliser.inverse_transform(y_predicted)
true_y_test = y_normaliser.inverse_transform(Y_test)

In [20]:
loss = np.mean((true_y_test - predicted_y_test)**2)
medium_error = np.mean(abs(true_y_test - predicted_y_test))
print(f'MSE is {loss:.2f}\nMedium error is {medium_error:.2f}')

MSE is 6429.10
Medium error is 5.63


In [21]:
print(f'Trend accuracy is: {(sum(1 for r,p in zip(predicted_y_test,true_y_test) if (r > 0 and p > 0) or (r < 0 and p < 0))/len(Y_test))*100:.2f}%')

Trend accuracy is: 95.49%


In [32]:
print('Test growth trends:        Predicted growth trends:')
print(sum(1 for v in true_y_test if v > 0),'                     ',sum(1 for v in predicted_y_test if v > 0))
print('Test decreasing trends:    Predicted decreasing trends:')
print(sum(1 for v in true_y_test if v < 0),'                     ',sum(1 for v in predicted_y_test if v < 0))

Test growth trends:        Predicted growth trends:
3110                       2885
Test decreasing trends:    Predicted decreasing trends:
2855                       3080


In [34]:
print(f'Test mean: {np.mean(true_y_test)}')
print(f'Predictions mean: {np.mean(predicted_y_test)}')

Test mean: 13.814097446110715
Predictions mean: 14.388855934143066


##### Results seem incredibly good!

In [60]:
print('Test values:    Predicted values:')
for r,p in zip(true_y_test.flatten(),predicted_y_test.flatten()):
    print(f'    {r:.2f}    {p:>12.2f}')

Test values:    Predicted values:
    6.71            4.72
    15.98           12.71
    -24.16          -21.00
    -3.26           -3.81
    -37.74          -32.02
    19.17           13.64
    -22.51          -19.86
    3.10            1.58
    -32.61          -27.95
    -2.50           -3.28
    -35.70          -30.71
    5.68            0.93
    5.03            3.37
    42.44           35.66
    4.46           -0.66
    -29.52          -25.52
    7.42            5.21
    -4.15           -7.50
    -16.67          -14.98
    37.96           31.92
    -21.83          -19.48
    -5.49           -5.83
    3.60            1.66
    17.70           14.24
    7.64            5.53
    19.82           16.10
    -11.20          -10.42
    4.49            2.79
    6.44            4.49
    -1.85           -2.61
    9.54            5.72
    0.44           -2.28
    17.55           14.06
    -31.37          -27.16
    10.42            8.26
    -25.96          -22.85
    5.77            3.78
    26

    13.12           10.20
    -70.48          -58.22
    42.77           36.25
    18.53           15.00
    -18.17          -16.40
    94.55           82.96
    12.27            9.46
    -15.63          -14.23
    35.45           29.75
    -27.61          -24.14
    -7.09           -7.05
    -26.67          -23.91
    -2.70           -3.63
    7.89            3.57
    17.42           13.70
    6.89            4.94
    -30.67          -26.55
    -6.67           -6.44
    5.23            3.27
    56.11           48.23
    -3.70           -4.31
    -16.70          -14.87
    -33.85          -28.88
    43.85           37.15
    3.59            1.62
    -5.66           -5.76
    -2.56           -3.17
    -3.17           -4.75
    -67.71          -55.74
    25.49           20.95
    -19.94          -17.61
    66.13           56.82
    11.77            8.96
    1.17           -0.06
    -28.07          -23.73
    11.94            9.26
    -37.65          -32.30
    0.79           -0.91
    17

    31.70           26.40
    -13.02          -12.01
    -2.37           -2.98
    8.29            5.34
    -31.79          -27.60
    10.83            7.19
    51.76           44.14
    2.85            1.36
    -11.01          -10.15
    -30.58          -26.67
    5.26            3.53
    2.26            0.91
    -13.77          -12.49
    6.19            1.71
    -10.92          -10.28
    -16.76          -14.91
    5.69            3.84
    13.24           10.27
    46.26           39.15
    -6.54           -6.43
    -6.72           -6.70
    1.37           -0.58
    10.54            8.02
    9.68            8.18
    -29.88          -25.90
    50.88           43.29
    4.27            3.85
    -14.18          -12.89
    13.89           10.80
    0.25          -22.46
    2.33            2.10
    -2.26           -2.96
    1016.33         1021.06
    -7.77           -7.45
    4.47            2.93
    -2.91           -3.53
    -56.17          -47.10
    -34.10          -29.44
    -1.08  